In [6]:
import json

with open('../../crawler_downloads/20220504_152756_products.jl', 'r') as file:
    products = list(map(lambda l: json.loads(l), file.readlines()))

In [7]:
len(products)

7954

In [12]:
a_ids = set(map(lambda p: p['articleId'], products))

In [8]:
import pandas as pd

products_df = pd.read_csv('./billa_shop_products.csv')
len(products_df['articleId'].unique())

9642

In [11]:
b_ids = set(products_df['articleId'].unique())

In [27]:
products_df[products_df['articleId'].isin(b_ids.difference(a_ids))].groupby('group').size().sort_values()

group
['B2-6', 'B2-63', 'B2-636']      1
['B2-6', 'B2-68', 'B2-684']      1
['B2-6', 'B2-68', 'B2-683']      1
['B2-6', 'B2-68', 'B2-681']      1
['B2-6', 'B2-67', 'B2-671']      1
                              ... 
['B2-3', 'B2-31', 'B2-314']     76
['B2-4', 'B2-45', 'B2-453']     90
['B2-6', 'B2-68', 'B2-686']     98
['B2-6', 'B2-6C', 'B2-6C3']     98
['B2-3', 'B2-33', 'B2-331']    168
Length: 101, dtype: int64

In [28]:
with open('./product_groups_flat.json', 'r') as file:
    product_groups = json.load(file)

In [43]:
set(map(lambda x: f'{x}', (map(lambda x: list(map(lambda y: y['id'], x['contained_in'])) + [x['id']], filter(lambda x: x['is_leaf'], product_groups.values())))))

{"['B2-1', 'B2-11', 'B2-111']",
 "['B2-1', 'B2-11', 'B2-112']",
 "['B2-1', 'B2-11', 'B2-113']",
 "['B2-1', 'B2-11', 'B2-114']",
 "['B2-1', 'B2-12', 'B2-121']",
 "['B2-1', 'B2-12', 'B2-122']",
 "['B2-1', 'B2-12', 'B2-123']",
 "['B2-1', 'B2-13', 'B2-131']",
 "['B2-1', 'B2-13', 'B2-132']",
 "['B2-1', 'B2-14']",
 "['B2-2', 'B2-21', 'B2-211']",
 "['B2-2', 'B2-21', 'B2-212']",
 "['B2-2', 'B2-22', 'B2-221']",
 "['B2-2', 'B2-22', 'B2-222']",
 "['B2-2', 'B2-22', 'B2-223']",
 "['B2-2', 'B2-23']",
 "['B2-2', 'B2-24', 'B2-241']",
 "['B2-2', 'B2-24', 'B2-242']",
 "['B2-2', 'B2-24', 'B2-243']",
 "['B2-2', 'B2-24', 'B2-244']",
 "['B2-2', 'B2-24', 'B2-245']",
 "['B2-2', 'B2-24', 'B2-246']",
 "['B2-2', 'B2-24', 'B2-247']",
 "['B2-2', 'B2-25']",
 "['B2-3', 'B2-31', 'B2-311']",
 "['B2-3', 'B2-31', 'B2-312']",
 "['B2-3', 'B2-31', 'B2-313']",
 "['B2-3', 'B2-31', 'B2-314']",
 "['B2-3', 'B2-31', 'B2-315']",
 "['B2-3', 'B2-31', 'B2-316']",
 "['B2-3', 'B2-31', 'B2-317']",
 "['B2-3', 'B2-31', 'B2-318']",
 "['B2

In [33]:
len(products_df['group'].unique())

294

In [44]:
with open('./productGroups_2022_05_01.json', 'r') as file:
    orig_groups = json.load(file)

In [67]:
def product_groups_list(product_groups, parent_groups=[]):

    result = []
    for product_group_name, product_group in product_groups.items():
        result.append({
            'id': product_group['id'],
            'name': product_group_name,
            'contained_in': parent_groups,
            'is_leaf': product_group['subGroups'] is None
        })
        
        if product_group['subGroups'] is not None:
            result.extend(product_groups_list(product_group['subGroups'], parent_groups + [ product_group['id'] ]))
            
    return result


In [68]:
product_group_df = pd.DataFrame(product_groups_list(orig_groups)).set_index('id')

In [69]:
product_group_df.index.is_unique

True

In [70]:
product_group_df.to_csv('./product_groups.csv')

In [80]:
with open('./smallest_product_groups.json', 'w') as file:
    json.dump(product_group_df.query('is_leaf == True').index.unique().values.tolist(), file)